# Pandas基础

该部分主要参考[Pandas中文网](https://www.pypandas.cn/docs/)和[官方用户手册](https://pandas.pydata.org/pandas-docs/stable/index.html)。

Pandas是一个开源的，为Python编程语言提供高性能，易于使用的**数据结构**和**数据分析工具**。在hydrus环境下安装pandas：

```Shell
conda install -c conda-forge pandas
```

因此这部分的记录主要思路就是围绕数据结构和分析工具两方面来展开。两部分之间会有重复的地方，就当是复习了。

首先，简单记录下关于pandas的介绍。Pandas适合处理多种不同类型的数据：

- 不同类型的数据列组成的表格数据；
- 有序或无序的时间序列数据；
- 带行列标签的类型相同或不同的行或列组成的矩阵数据；
- 任意形式的观测或统计数据，它们不需标记即可放入pandas数据结构中。 

其运算都主要都围绕两类数据结构展开，Series和Dataframe，接下来从它们开始记录。

Series(1维)和DataFrame(2维)能处理绝大多数的典型应用，包括金融、统计、社科、工程等各个领域。
 
对于R用户，DataFrame提供了所有R的data.frame提供的，并且还有更多。

pandas建立在NumPy之上，也整合了很多第三方库，打造了良好的科学计算环境。

pandas处理的经典问题包括且不限于：

- 处理缺失数据，用NaN表示；
- 数据结构大小可变：可以插入删除DataFrame或者更高维对象的列；
- 自动且明确的data alignment，可以明确指定数据到一系列标签下；
- 灵活数据分组，很容易聚合转换数据；
- 容易将杂乱、标签复杂的Python和NumPy数据结构转换为DataFrame对象；
- 根据标签容易slicing,indexing和subsetting大数据集；
- 容易合并和连接数据集；
- 灵活地reshaping和pivoting数据集；
- 多级标签；
- 从各类文件中加载数据, 包括HDF5格式的数据；
- 时间序列指定的功能：日期范围生成和频率转换,滑动窗口的统计, 滑动线性回归, 日期平移或滞后等。

在数据处理的各个环节，包括变换和清洗数据，分析建模，组织分析结果为正式的格式，可视化和输出数据等，pandas都能很好地处理。

## Pandas数据结构

最好的理解Pandas数据结构的方式是将其当做其更低维数据的容器，比如Series是标量数据的容器，Dataframe是Series的容器。
可以通过类似字典的方式从容器中插入或移除对象。

pandas中为了更直观地给出数据结构不同维度的信息，更多用index和columns，而不是使用axis0和1这样来表示。例如：

``` python
for col in df.columns:
    series = df[col]
    # do something with series
```

pandas所有数据结构都是值可变的，但不总是size可变的。比如Series的长度是不可变的，但是Dataframe中是可以插入列的。
绝大多数方法都会产生新的对象，而不改变原输入数据。一般来说，pandas喜欢保持不可变性。

### Series与Dataframe

对于pandas的这两个数据结构，个人认为从增删改查的角度了解较好，首先是数据结构的初始化，然后增删操作，包括拼接操作等，接下来就是修改名称等，而后重点关注索引切片等查询操作，最后再补充一些分组等其他内容。

#### 初始化

基本上来说，是利用numpy的数组为值依据，用columns和index给列和行起名。

In [7]:
import numpy as np
import pandas as pd
# 初始化Series
k = pd.Series({'a':np.random.randint(10,size=5)})
k

a    [1, 2, 1, 8, 2]
dtype: object

In [8]:
# 创建一个空的 DataFrame
df_empty = pd.DataFrame(columns=['A', 'B', 'C', 'D'])
print(df_empty)
print("列名：",df_empty.columns.values)
df = pd.DataFrame({"a": range(3), "b": range(3), "c": range(3)})
print(df)
# 使用numpy数组，命名行列进行初始化
df2 = pd.DataFrame(np.arange(16).reshape((4,4)), columns=['one', 'two', 'three', 'four'], index=['a', 'b', 'c','d'])
print(df2)
# 带时间序列的dataframe的初始化
time_range = pd.date_range('2011-1-1', periods=4, freq='D')
df2.index = time_range
print(df2)
print('列：\n',df2.columns)
print('列名称：\n',df2.columns.values)
time_range = pd.date_range('2011-1-1', '2011-1-4', freq='D')
df2.columns = time_range
print(df2)

Empty DataFrame
Columns: [A, B, C, D]
Index: []
列名： ['A' 'B' 'C' 'D']
   a  b  c
0  0  0  0
1  1  1  1
2  2  2  2
   one  two  three  four
a    0    1      2     3
b    4    5      6     7
c    8    9     10    11
d   12   13     14    15
            one  two  three  four
2011-01-01    0    1      2     3
2011-01-02    4    5      6     7
2011-01-03    8    9     10    11
2011-01-04   12   13     14    15
列：
 Index(['one', 'two', 'three', 'four'], dtype='object')
列名称：
 ['one' 'two' 'three' 'four']
            2011-01-01  2011-01-02  2011-01-03  2011-01-04
2011-01-01           0           1           2           3
2011-01-02           4           5           6           7
2011-01-03           8           9          10          11
2011-01-04          12          13          14          15


有时候会需要指定某些列的数据类型，这时候可以使用如下方式：

In [22]:
a = [['a', '1.2', '4.2'], ['b', '70', '0.03'], ['x', '5', '0']]
df = pd.DataFrame(a, columns=['one', 'two', 'three'])
df

,one,two,three
0,a,1.2,4.2
1,b,70,0.03
2,x,5,0


In [23]:
df.dtypes

one      object
two      object
three    object
dtype: object

In [27]:
df[['two', 'three']] = df[['two', 'three']].astype(float)
df.dtypes

one       object
two      float64
three    float64
dtype: object

#### 日期时间处理

和python基础，numpy一样，pandas也有自己的日期处理包，日期总是比较麻烦的。

In [5]:
import pandas as pd
# 日期／字符串转换
strtime=['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-30', '2000-10-31']
index=pd.to_datetime(strtime)
time=pd.to_datetime(strtime)
[[dt.year, dt.month, dt.day, dt.hour] for dt in time]

[[2000, 1, 31, 0],
 [2000, 2, 29, 0],
 [2000, 3, 31, 0],
 [2000, 4, 30, 0],
 [2000, 5, 31, 0],
 [2000, 6, 30, 0],
 [2000, 7, 31, 0],
 [2000, 8, 31, 0],
 [2000, 9, 30, 0],
 [2000, 10, 31, 0]]

In [9]:
my_time = pd.DataFrame([[dt.year, dt.month, dt.day, dt.hour] for dt in time], columns=['Year', 'Mnth', 'Day', 'Hr'])
my_time.head()

,Year,Mnth,Day,Hr
0,2000,1,31,0
1,2000,2,29,0
2,2000,3,31,0
3,2000,4,30,0
4,2000,5,31,0


In [4]:
data=list(range(1,11))
df=pd.DataFrame([time,data],columns=index,index=['time','data']).T

#索引和time字段均为Timestamp格式数据
print(type(index[0]))

print(type(df.index[0]))

print(type(df.time[0]))


#先来看索引
mt_time=df.index.strftime('%Y-%m')
mt_time

print(type(mt_time[0]))

#针对列（直接操作还不清楚怎么操作）
time_list=[x.strftime('%Y-%m') for x in df['time']]
print(time_list)
#再按月份求和
df['data'].groupby(time_list).sum()

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'str'>
['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06', '2000-07', '2000-08', '2000-09', '2000-10']


2000-01     1
2000-02     2
2000-03     3
2000-04     4
2000-05     5
2000-06     6
2000-07     7
2000-08     8
2000-09     9
2000-10    10
Name: data, dtype: int64

再比如下面的数据形式，前面三列是年月日，最后一列是数据：

In [6]:
# 数字，日期转换
import pandas as pd
data_temp = pd.DataFrame([[1952,1,1,3950],
[1952,1,2,3920],
[1952,1,3,3960],
[1952,1,4,3970]])
data_temp

,0,1,2,3
0,1952,1,1,3950
1,1952,1,2,3920
2,1952,1,3,3960
3,1952,1,4,3970


也很容易将数字转为日期：

In [7]:
df_date = data_temp[[0, 1, 2]]
print(df_date)
df_date.columns = ['year', 'month', 'day']
date = pd.to_datetime(df_date).values.astype('datetime64[D]')
date

      0  1  2
0  1952  1  1
1  1952  1  2
2  1952  1  3
3  1952  1  4


array(['1952-01-01', '1952-01-02', '1952-01-03', '1952-01-04'],
      dtype='datetime64[D]')

#### 排序

对DataFrame中的数据进行排序也是常用的操作之一。比如：

In [17]:
import pandas as pd
import numpy as np
df = pd.DataFrame({
    'col1': ['A', 'A', 'B', np.nan, 'D', 'C'],
    'col2': [2, 1, 9, 8, 7, 4],
    'col3': [0, 1, 9, 4, 2, 3],
})
df

,col1,col2,col3
0,A,2,0
1,A,1,1
2,B,9,9
3,NaN,8,4
4,D,7,2
5,C,4,3


Sort by col1

In [20]:
df.sort_values(by=['col1'])

,col1,col2,col3
0,A,2,0
1,A,1,1
2,B,9,9
5,C,4,3
4,D,7,2
3,NaN,8,4


Sort by multiple columns

In [21]:
df.sort_values(by=['col1', 'col2'])

,col1,col2,col3
1,A,1,1
0,A,2,0
2,B,9,9
5,C,4,3
4,D,7,2
3,NaN,8,4


#### 增删，拼接等操作

可参考：[Merge, join, and concatenate](https://www.pypandas.cn/docs/user_guide/merging.html#concatenating-objects)

拼接主要使用的是concat函数，axis=0是默认选项纵向拼接，axis=1是横向

In [10]:
import pandas as pd
df0 = pd.DataFrame()
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                   'B': ['B0', 'B1', 'B2', 'B3'],
                        'C': ['C0', 'C1', 'C2', 'C3'],
                        'D': ['D0', 'D1', 'D2', 'D3']},
                     index=[0, 1, 2, 3]) 

df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                   'B': ['B4', 'B5', 'B6', 'B7'],
                  'C': ['C4', 'C5', 'C6', 'C7'],
                     'D': ['D4', 'D5', 'D6', 'D7']},
                    index=[4, 5, 6, 7]) 

df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                     'B': ['B8', 'B9', 'B10', 'B11'],
                    'C': ['C8', 'C9', 'C10', 'C11'],
                    'D': ['D8', 'D9', 'D10', 'D11']},
                    index=[8, 9, 10, 11]) 

frames = [df0, df1, df2, df3]
result = pd.concat(frames)
print(result)

      A    B    C    D
0    A0   B0   C0   D0
1    A1   B1   C1   D1
2    A2   B2   C2   D2
3    A3   B3   C3   D3
4    A4   B4   C4   D4
5    A5   B5   C5   D5
6    A6   B6   C6   D6
7    A7   B7   C7   D7
8    A8   B8   C8   D8
9    A9   B9   C9   D9
10  A10  B10  C10  D10
11  A11  B11  C11  D11


In [11]:
df4 = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'],
                        'D': ['D2', 'D3', 'D6', 'D7'],
                        'F': ['F2', 'F3', 'F6', 'F7']},
                       index=[2, 3, 6, 7])
result = pd.concat([df1, df4], axis=1)
result

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3
6,NaN,NaN,NaN,NaN,B6,D6,F6
7,NaN,NaN,NaN,NaN,B7,D7,F7


再看 一个增加空列的例子，主要参考了：[pandas dataframe在指定的位置添加一列, 或者一次性添加几列，reindex，pd.concat的使用](https://blog.csdn.net/AlanGuoo/article/details/76522429)

In [13]:
df5 = pd.concat([df4, pd.DataFrame(columns=['flow', 'mode'])])
df5

,B,D,F,flow,mode
2,B2,D2,F2,NaN,NaN
3,B3,D3,F3,NaN,NaN
6,B6,D6,F6,NaN,NaN
7,B7,D7,F7,NaN,NaN


reindex可以来重新索引，如下所示，只用了df1的index，所以就剩下4行了

In [17]:
df5.reindex(df1.index)

,B,D,F,flow,mode
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,B2,D2,F2,NaN,NaN
3,B3,D3,F3,NaN,NaN


另外还有一个多列拼接成一列的小例子，几列字符串拼为一列字符串：

In [13]:
df = pd.DataFrame({"a": range(3), "b": range(3), "c": range(3)})
# 结构很简单: 第一列的名称.str.cat(第二列的名称)
df['a'] = df.iloc[:, 0].apply(str) + "-" + df['b'].apply(str) + "-" + df['b'].apply(str)
# 拼接之后，只留下特定的几列
df = df[['a', 'c']]
print(df)

       a  c
0  0-0-0  0
1  1-1-1  1
2  2-2-2  2


多个Series拼接

In [2]:
import pandas as pd
# 初始化的时候不起名，后面rename没有用
a = pd.Series([1, 2], name='aa')
rng1 = pd.date_range('2011-1-1', periods=2, freq='D')
print(type(rng1))
a.index = rng1
print("构建一个序列，并以时间做index：",a)
b = pd.Series([2, 3, 4])
rng2 = pd.date_range('2011-1-2', periods=3, freq='D')
b.index = rng2
df1 = pd.concat([a, b], axis=1)
print("拼接a和b：",df1)
c = pd.Series([5, 6])
rng3 = pd.date_range('2011-1-1', periods=2, freq='D')
c.index = rng3
df2 = pd.concat([df1, c], axis=1)
print("拼接df1和c：",df2)
# 尝试直接拼接多个：
df3 = pd.concat([df1, c, c], axis=1)
print("拼接多个目标：",df3)

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
构建一个序列，并以时间做index： 2011-01-01    1
2011-01-02    2
Freq: D, Name: aa, dtype: int64
拼接a和b：              aa    0
2011-01-01  1.0  NaN
2011-01-02  2.0  2.0
2011-01-03  NaN  3.0
2011-01-04  NaN  4.0
拼接df1和c：              aa    0    0
2011-01-01  1.0  NaN  5.0
2011-01-02  2.0  2.0  6.0
2011-01-03  NaN  3.0  NaN
2011-01-04  NaN  4.0  NaN
拼接多个目标：              aa    0    0    1
2011-01-01  1.0  NaN  5.0  5.0
2011-01-02  2.0  2.0  6.0  6.0
2011-01-03  NaN  3.0  NaN  NaN
2011-01-04  NaN  4.0  NaN  NaN


拼接还可以使用append：

In [15]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                        'B': ['B0', 'B1', 'B2', 'B3'],
                        'C': ['C0', 'C1', 'C2', 'C3'],
                        'D': ['D0', 'D1', 'D2', 'D3']},
                       index=[0, 1, 2, 3])
df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                        'B': ['B4', 'B5', 'B6', 'B7'],
                        'C': ['C4', 'C5', 'C6', 'C7'],
                        'D': ['D4', 'D5', 'D6', 'D7']},
                       index=[4, 5, 6, 7])
result = df1.append(df2)
result

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


#### 修改操作

包括修改行列名，交换行列等操作。

In [16]:
"""重新命名各列"""
new_col = ['new1', 'new2', 'new3', 'new4']
df2.columns = new_col
print(df2)

"""交换列的位置"""
order = ['new2', 'new1', 'new3', 'new4']
df2 = df2[order]
print(df2)

  new1 new2 new3 new4
4   A4   B4   C4   D4
5   A5   B5   C5   D5
6   A6   B6   C6   D6
7   A7   B7   C7   D7
  new2 new1 new3 new4
4   B4   A4   C4   D4
5   B5   A5   C5   D5
6   B6   A6   C6   D6
7   B7   A7   C7   D7


行列拆分，参考[DataFrame一列拆成多列以及一行拆成多行](https://blog.csdn.net/Asher117/article/details/84346073)，在数据分析时，经常需要把DataFrame的一列拆成多列或者根据某列把一行拆成多行。

In [17]:
import pandas as pd

df= pd.DataFrame({'Country': ['China', 'America', 'Japan'],
                   'City': ['Shanghai|Shenzhen', 'New York|State College', 'Tokyo|Osaka']},
                     index=[0, 1, 2])
df

,Country,City
0,China,Shanghai|Shenzhen
1,America,New York|State College
2,Japan,Tokyo|Osaka


In [18]:
df['Citys']=df['City'].map(lambda x: x.split('|'))
df

,Country,City,Citys
0,China,Shanghai|Shenzhen,"[Shanghai, Shenzhen]"
1,America,New York|State College,"[New York, State College]"
2,Japan,Tokyo|Osaka,"[Tokyo, Osaka]"


In [19]:
df['City1']=df['City'].map(lambda x: x.split('|')[0])
df['City2']=df['City'].map(lambda x: x.split('|')[1])
df

,Country,City,Citys,City1,City2
0,China,Shanghai|Shenzhen,"[Shanghai, Shenzhen]",Shanghai,Shenzhen
1,America,New York|State College,"[New York, State College]",New York,State College
2,Japan,Tokyo|Osaka,"[Tokyo, Osaka]",Tokyo,Osaka


除了上面的拆分，还有将各列拼接到一起，重新组织成符合数据库范式形式的表格，这就要用到十分常用的melt函数了，转换后的格式：其中一个或多个列是标识符变量，而所有其他列(被认为是测量变量)都不以行轴为轴，**只留下两个非标识符列**(变量和值)。

In [20]:
import pandas as pd
df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},
                    'B': {0: 1, 1: 3, 2: 5},
                    'C': {0: 2, 1: 4, 2: 6}})
df

,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6


In [21]:
pd.melt(df, id_vars=['A'], value_vars=['B'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5


In [22]:
pd.melt(df, id_vars=['A'], value_vars=['B', 'C'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5
3,a,C,2
4,b,C,4
5,c,C,6


In [23]:
pd.melt(df, id_vars=['A'], value_vars=['B'], var_name='myVarname', value_name='myValname')

,A,myVarname,myValname
0,a,B,1
1,b,B,3
2,c,B,5


#### 索引，切片等操作

首先是直接[]索引，以及loc和iloc的使用，这是pandas中最常用的索引方式。

In [6]:
import numpy as np
import pandas as pd

ser = pd.Series(np.arange(3.))

data = pd.DataFrame(np.arange(16).reshape(4,4),index=list('abcd'),columns=list('wxyz'))
print(data)
print("选择表格中的'w'列，使用类字典属性,返回的是Series类型:", data['w'])

print("选择表格中的'w'列，使用点属性,返回的是Series类型:",data.w)

print("选择表格中的'w'列，返回的是DataFrame属性:",data[['w']])

print("选择表格中的'w'、'z'列:",data[['w','z']])  

print("返回第1行到第2行的所有行，前闭后开，包括前不包括后:",data[0:2])

print(" #返回第2行，从0计，返回的是单行，通过有前后值的索引形式:",data[1:2]) 
       #如果采用data[1]则报错

print("#返回第2行的第三种方法，返回的是DataFrame，跟data[1:2]同:",data.iloc[1:2]) 

print(" #利用index值进行切片，返回的是**前闭后闭**的DataFrame:",data['a':'b']) 

print("#返回data的前几行数据，默认为前五行，需要前十行则dta.head(10):",data.head())  
print("#返回data的后几行数据，默认为后五行，需要后十行则data.tail(10):",data.tail())  


print("#选取DataFrame最后一行，返回的是Series:",data.iloc[-1])   
print("#选取DataFrame最后一行，返回的是DataFrame:",data.iloc[-1:])  

print("#返回‘a’行'w'、'x'列，这种用于选取行索引列索引已知",data.loc['a',['w','x']])   

print("#选取第二行第二列，用于已知行、列位置的选取:",data.iat[1,1])   

    w   x   y   z
a   0   1   2   3
b   4   5   6   7
c   8   9  10  11
d  12  13  14  15
选择表格中的'w'列，使用类字典属性,返回的是Series类型: a     0
b     4
c     8
d    12
Name: w, dtype: int32
选择表格中的'w'列，使用点属性,返回的是Series类型: a     0
b     4
c     8
d    12
Name: w, dtype: int32
选择表格中的'w'列，返回的是DataFrame属性:     w
a   0
b   4
c   8
d  12
选择表格中的'w'、'z'列:     w   z
a   0   3
b   4   7
c   8  11
d  12  15
返回第1行到第2行的所有行，前闭后开，包括前不包括后:    w  x  y  z
a  0  1  2  3
b  4  5  6  7
 #返回第2行，从0计，返回的是单行，通过有前后值的索引形式:    w  x  y  z
b  4  5  6  7
#返回第2行的第三种方法，返回的是DataFrame，跟data[1:2]同:    w  x  y  z
b  4  5  6  7
 #利用index值进行切片，返回的是**前闭后闭**的DataFrame:    w  x  y  z
a  0  1  2  3
b  4  5  6  7
#返回data的前几行数据，默认为前五行，需要前十行则dta.head(10):     w   x   y   z
a   0   1   2   3
b   4   5   6   7
c   8   9  10  11
d  12  13  14  15
#返回data的后几行数据，默认为后五行，需要后十行则data.tail(10):     w   x   y   z
a   0   1   2   3
b   4   5   6   7
c   8   9  10  11
d  12  13  14  15
#选取DataFrame最后一行，返回的是Series: w    12
x    13
y    14
z    15
Name: d, dt

如果某列用字符串索引，行用数字索引，那么需要这样做：

In [8]:
data = pd.DataFrame(np.arange(16).reshape(4,4),index=list('abcd'),columns=list('wxyz'))
print(data)
data['x'][0]

    w   x   y   z
a   0   1   2   3
b   4   5   6   7
c   8   9  10  11
d  12  13  14  15


1

In [29]:
"""取dataframe指定多行列 slice操作"""
# 取多行
print(df2.iloc[0:2])
# 取多列
print(df2.iloc[:, 0:2])
# 取多行多列
print(df2.iloc[0:2, 0:2])

             aa    0    0
2011-01-01  1.0  NaN  5.0
2011-01-02  2.0  2.0  6.0
             aa    0
2011-01-01  1.0  NaN
2011-01-02  2.0  2.0
2011-01-03  NaN  3.0
2011-01-04  NaN  4.0
             aa    0
2011-01-01  1.0  NaN
2011-01-02  2.0  2.0


series的slice

In [30]:
"""给Series作slice操作"""
arr = [1, 2, 3, 4]  # 创建数组
series_1 = pd.Series(arr)
series_1.index = ['a', 'b', 'c', 'd']
print("------------------Series查询操作----------------------")
print(series_1['a'])
print(series_1[['a', 'b']])
print(series_1[series_1 > 2])
print(series_1[:2])
print(series_1['a':'c'])

------------------Series查询操作----------------------
1
a    1
b    2
dtype: int64
c    3
d    4
dtype: int64
a    1
b    2
dtype: int64
a    1
b    2
c    3
dtype: int64


在dataframe中还常用到条件筛选。参考[30分钟带你入门数据分析工具 Pandas（上篇），果断收藏](https://zhuanlan.zhihu.com/p/44174554)，用中括号 [] 的方式，除了直接指定选中某些列外，还能接收一个条件语句，然后筛选出符合条件的行/列。比如：

In [31]:
import pandas as pd
import numpy as np
df=pd.DataFrame(np.random.randn(5,4),['A','B','C','D','E'],['W','X','Y','Z'])
# 筛选出 'W'>0 的行
print(df[df['W']>0])
print('\n')
# 只看 'X' 列中 'W'>0 的数据
print(df[df['W']>0]['X'])
print('\n')
print(df[df['W']>0][['X','Y']])

          W         X         Y         Z
E  0.805262  1.495269  0.997251  0.830787


E    1.495269
Name: X, dtype: float64


          X         Y
E  1.495269  0.997251


还可以用逻辑运算符 &（与）和 |（或）来链接多个条件语句，以便一次应用多个筛选条件到当前的 DataFrame 上，比如筛选出同时满足 'W'>0 和'X'>1 的行：

In [32]:
df[(df['W']>0) & (df['X']>1)]

,W,X,Y,Z
E,0.805262,1.495269,0.997251,0.830787


再比如，还可以让列之间进行比较并作为索引条件

In [33]:
df

,W,X,Y,Z
A,-0.673042,-2.280204,0.684003,-1.526174
B,-0.940006,0.390133,0.145146,2.580225
C,-1.401806,-0.699417,-1.418799,-0.127906
D,-0.043496,1.327835,0.276914,0.787950
E,0.805262,1.495269,0.997251,0.830787


In [34]:
df[df['W']<df['X']]

,W,X,Y,Z
B,-0.940006,0.390133,0.145146,2.580225
C,-1.401806,-0.699417,-1.418799,-0.127906
D,-0.043496,1.327835,0.276914,0.787950
E,0.805262,1.495269,0.997251,0.830787


使用pandas的loc时要注意，当有missing labels，在pandas 1.0版本之后是会报错的，更推荐的做法是使用reindex()

In [2]:
import pandas as pd
s = pd.Series([1, 2, 3])
s.loc[[1, 2]]

1    2
2    3
dtype: int64

In [3]:
s.loc[[1, 2, 3]] # missing value使用loc会报错

KeyError: 'Passing list-likes to .loc or [] with any missing labels is no longer supported, see https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike'

现在看看使用reindex，更多内容可以参考：https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike

In [4]:
s.reindex([1, 2, 3])

1    2.0
2    3.0
3    NaN
dtype: float64

如果仅仅想要去除valid keys对应的值，可以使用：

In [6]:
labels = [1, 2, 3]
s.index.intersection(labels)
# s.loc[s.index.intersection(labels)]

Int64Index([1, 2], dtype='int64')

#### 分组

对dataframe进行分组的例子。

In [35]:
salaries = pd.DataFrame({
    'name': ['BOSS', 'Lilei', 'Lilei', 'Han', 'BOSS', 'BOSS', 'Han', 'BOSS'],
    'Year': [2016, 2016, 2016, 2016, 2017, 2017, 2017, 2017],
    'Salary': [999999, 20000, 25000, 3000, 9999999, 999999, 3500, 999999],
    'Bonus': [100000, 20000, 20000, 5000, 200000, 300000, 3000, 400000]
})
print(salaries.columns)
print(salaries.info())
print(salaries.describe())
salaries = salaries[['name', 'Year', 'Salary', 'Bonus']]
# 定顺序
print(salaries)
# 对dataframe按name进行分组
group_by_name = salaries.groupby('name')
# 获取分组后的某一组
se_temp = group_by_name.get_group('Lilei')
print(se_temp)
print(se_temp.describe())
# 循环各组，并将名字在给定的序列中的group添加到数组中
config = pd.Series({'names': ['Lilei', 'BOSS']})
dfs = []
for name, group in group_by_name:
    print(name)
    if name in config['names']:
        dfs.append(group)

for i in range(len(dfs)):
    # 如果没有把name和group分开，那么dfs[i]的类型会是tuple，key为name，value是group，可参考接下来的输出
    print(type(dfs[i]))
    print(dfs[i])

# 如果没有把name和group分开，那么dfs[i]的类型会是tuple，key为name，value是group
dfs_s = []
for group in group_by_name:
    dfs_s.append(group)
for i in range(len(dfs_s)):
    print(type(dfs_s[i]))
    print(dfs_s[i])

Index(['name', 'Year', 'Salary', 'Bonus'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    8 non-null      object
 1   Year    8 non-null      int64 
 2   Salary  8 non-null      int64 
 3   Bonus   8 non-null      int64 
dtypes: int64(3), object(1)
memory usage: 384.0+ bytes
None
              Year        Salary          Bonus
count     8.000000  8.000000e+00       8.000000
mean   2016.500000  1.631437e+06  131000.000000
std       0.534522  3.416521e+06  152851.935826
min    2016.000000  3.000000e+03    3000.000000
25%    2016.000000  1.587500e+04   16250.000000
50%    2016.500000  5.124995e+05   60000.000000
75%    2017.000000  9.999990e+05  225000.000000
max    2017.000000  9.999999e+06  400000.000000
    name  Year   Salary   Bonus
0   BOSS  2016   999999  100000
1  Lilei  2016    20000   20000
2  Lilei  2016    25000   20000
3    

### pandas与numpy数据结构之间的转换

pandas转为numpy比较简单，直接调用value即转为ndarray。反过来，也很容易，直接用pandas的类型，相当于直接初始化了。

In [36]:
# pandas dataframe与numpy array之间的转换
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
print(df)
# df转换为ndarray
array_from_df=df.values
print(array_from_df)
print(np.array(df))
# 读取某一列数据，两种方式均可
print(df['A'])
print(df.loc[:, 'A'])
# 一列数据转换为ndarray
print(np.array(df['A']))
# Series转ndarray
# Creating the Series
sr = pd.Series(['New York', 'Chicago', 'Toronto', 'Lisbon', 'Rio'])
# Create the Index
index_ = ['City 1', 'City 2', 'City 3', 'City 4', 'City 5']
# set the index
sr.index = index_
# return numpy array representation
result = sr.values
# Print the result
print("series转为ndarray：")
print(result)
# Print the series
print(sr)

   A  B  C
0  1  4  7
1  2  5  8
2  3  6  9
[[1 4 7]
 [2 5 8]
 [3 6 9]]
[[1 4 7]
 [2 5 8]
 [3 6 9]]
0    1
1    2
2    3
Name: A, dtype: int64
0    1
1    2
2    3
Name: A, dtype: int64
[1 2 3]
series转为ndarray：
['New York' 'Chicago' 'Toronto' 'Lisbon' 'Rio']
City 1    New York
City 2     Chicago
City 3     Toronto
City 4      Lisbon
City 5         Rio
dtype: object


## Pandas数据分析

从数据读取、处理、分析和可视化及输出几个方面展开。

### IO工具

对于csv文件和txt文件，都可以通过pandas的read_csv()函数读取。

In [1]:
from io import StringIO, BytesIO
import pandas as pd
data = ('col1,col2,col3\n'
         'a,b,01\n'
         'a,b,02\n'
         'c,d,03')
print("原数据：\n",data)
print("pandas读取默认设置：\n",pd.read_csv(StringIO(data)))
# 读取数据的时候即进行slice
print("pandas读取前两列：\n",pd.read_csv(StringIO(data), usecols=range(0,2)))
print("pandas读取某两列：\n",pd.read_csv(StringIO(data), usecols=lambda x: x.upper() in ['COL1', 'COL3']))
print("pandas读取某些行：\n",pd.read_csv(StringIO(data), skiprows=lambda x: x % 2 != 0))
print("读取数据之后，重新命名列名：\n")
df_example=pd.read_csv(StringIO(data))
df_example.columns = ['A','B','C']
df_example

原数据：
 col1,col2,col3
a,b,01
a,b,02
c,d,03
pandas读取默认设置：
   col1 col2  col3
0    a    b     1
1    a    b     2
2    c    d     3
pandas读取前两列：
   col1 col2
0    a    b
1    a    b
2    c    d
pandas读取某两列：
   col1  col3
0    a     1
1    a     2
2    c     3
pandas读取某些行：
   col1 col2  col3
0    a    b     2
读取数据之后，重新命名列名：



,A,B,C
0,a,b,1
1,a,b,2
2,c,d,3


专业方面，很多数据都是把年月或者年旬或者年日分别当做行和列，在计算时，需要进行处理，把时间变为一列或一行，即把几列或行的数组拼接起来
即将[1 2;2 3]的数据变为[1 2 2 3]，pandas的concat和numpy的concatenate类似。最后把行名index统一换成日期，构成时间序列Series

In [6]:
# 如果遇到“UnicodeDecodeError: 'utf8' codec can't decode byte....”错误，用记事本另存csv文件时，将“编码”设置为‘UTF-8’即可
dataset = pd.read_csv('Sheet1.csv')
# header指定某行的值作为各列的列名，1表示第二行，如果是None，则从第一行开始就是数据。
dataset = pd.read_csv('Sheet1.csv', header=1)
# 因为给定的数据格式，各人有各自的一套，所以具体情况具体分析。比如，这里dataset的第一列可以去掉，即得到所有数据，删除列时，用drop函数，加参数axis=1，不加则表示删除行
dataset1 = dataset.drop(['旬平均'], axis=1)
df = pd.DataFrame({"a": range(3), "b": range(3), "c": range(3)})
# iloc获取的数据格式为Series
se = df.iloc[:, 0]
# 获取dataframe的列数：df.shape[1]
for i in range(1, df.shape[1]):
    se_temp = df.iloc[:, i]
    se = pd.concat([se, se_temp])
# 如果一开始没有给series起名，比如从csv读取出来时是dataframe，拼接之后没办法再给Series起名，但是又需要给它起名，那么只能采取重新构造一个series的方式来命名
se = pd.Series(se, name="aaaaaaa")
# 把每行名称换为日期
rng = pd.date_range('2011-1-1', periods=9, freq='H')
se.index = rng
se = pd.Series(se, name="bbbbb")
se.head()

2011-01-01 00:00:00    0
2011-01-01 01:00:00    1
2011-01-01 02:00:00    2
2011-01-01 03:00:00    0
2011-01-01 04:00:00    1
Freq: H, Name: bbbbb, dtype: int64

当csv中包含了大量的编号代码，比如是002开头的编号——002111， 使用pd.read_csv('text.csv') 则会让所有的002xxx，变成了2xxx，前面2个0不见了，因为作为数值类型，没有必要保留0。

因此，为了保持编号的字符串特性，直接在参数一栏设置一下即可：
df＝pd.read_csv('text.csv', dtype={'code':str}
 
这样，把你要转换的列的名字设定好， “code”列中的数据读取为str，用列名或者列序号均可。
 
这样，读取到的数据就是按照我们的要求的了。

In [39]:
dataset = pd.read_csv(StringIO(data),dtype={2:str})
print(dataset)
dataset = pd.read_csv(StringIO(data),dtype={'col3':str})
print(dataset)

  col1 col2 col3
0    a    b   01
1    a    b   02
2    c    d   03
  col1 col2 col3
0    a    b   01
1    a    b   02
2    c    d   03


读取excel文件的话稍微有些不同，这时候我们需要使用read_excel，第一次使用可能会遇到缺少 xlrd 包的情况，因此安装此包：

```Shell
conda install -c conda-forge xlrd
```

如果实在安装不成功，可以直接试试 pip install xlrd

In [5]:
import pandas as pd
# df = pd.read_excel('AK_U.xlsx',sheet_name='AK')
df = pd.read_excel('NID2018_U.xlsx')
df.head()

,RECORDID,DAM_NAME,OTHER_DAM_NAME,DAM_FORMER_NAME,STATEID,NIDID,LONGITUDE,LATITUDE,SECTION,COUNTY,...,CONG_NAME,PARTY,CONG_DIST,OTHERSTRUCTUREID,NUMSEPARATESTRUCTURES,PERMITTINGAUTHORITY,INSPECTIONAUTHORITY,ENFORCEMENTAUTHORITY,JURISDICTIONALDAM,EAP_LAST_REV_DATE
0,1,COOPER LAKE,NaN,NaN,NaN,AK00001,-149.823147,60.433708,NaN,KENAI PENINSULA,...,Don Young (R),R,AK00,NaN,1.0,N,N,N,N,08-JAN-18
1,2,BLUE LAKE,NaN,NaN,NaN,AK00002,-135.191700,57.063300,"T55S, R64E, S35",SITKA,...,Don Young (R),R,AK00,NaN,1.0,N,N,N,N,27-DEC-17
2,3,SALMON CREEK,NaN,NaN,NaN,AK00003,-134.403608,58.341850,"T41S,R67E,S2",JUNEAU,...,Don Young (R),R,AK00,NaN,1.0,N,N,N,N,27-SEP-17
3,4,ANNEX CREEK,NaN,NaN,NaN,AK00004,-134.126578,58.326939,"T41S,R69E,S9",JUNEAU,...,Don Young (R),R,AK00,NaN,2.0,N,N,N,N,27-SEP-17
4,5,CRYSTAL LAKE,NaN,NaN,NaN,AK00005,-132.845500,56.600000,"T61S,R81E,S6",WRANGELL-PETERSBURG,...,Don Young (R),R,AK00,NaN,1.0,N,N,N,N,19-DEC-17



### Prepare Data

#### 修改名称

包括列名和行名，行一般又称index；列名也称field。

总体上可以分为在读数据时修改和读后修改两种。

具体的，行的修改方法有多种，参考：[重命名dataframe的index](https://blog.csdn.net/sinat_35930259/article/details/79872577)；列名的修改方法也有多种，参考[Pandas中修改DataFrame列名](http://www.voidcn.com/article/p-wycobfgd-bqs.html)

In [40]:
# 给出所有列名，这里类型是Index，可以直接转为list
print(dataset.columns)
columns=dataset.columns.tolist()
print(columns)
# 转换后，每个元素直接就是字符串了
print(type(columns[0]))
# 修改列名的常用方式,inplace为false的话，还需要赋值到dataset，因此直接设置为true
dataset.rename(columns={'col1':'a', 'col2':'b', 'col3':'c'}, inplace = True)
print(dataset)
# 修改index行名

Index(['col1', 'col2', 'col3'], dtype='object')
['col1', 'col2', 'col3']
<class 'str'>
   a  b   c
0  a  b  01
1  a  b  02
2  c  d  03


#### Slice

切片包括横向切，纵向切，即选取某些行，选取某些列的操作。dataframe的切片操作很容易，记住灵活运用其index的功能即可，第一维是行，第二维是列，定位使用loc，数值定位用iloc。

In [41]:
# 取行直接用index即可
print(dataset.loc[0:1,:])
print("注意因为行名是0-1-2，所以loc和iloc略有不同")
print(dataset.iloc[0:1,:])
# 取列用field
print("列操作")
print(dataset.loc[:,'c'])
columns=['a','c']
print(dataset.loc[:,columns])

   a  b   c
0  a  b  01
1  a  b  02
注意因为行名是0-1-2，所以loc和iloc略有不同
   a  b   c
0  a  b  01
列操作
0    01
1    02
2    03
Name: c, dtype: object
   a   c
0  a  01
1  a  02
2  c  03


#### 数据类型转换

参考[Pandas数据类型转换的几个小技巧](https://zhuanlan.zhihu.com/p/35287822)

Pandas中进行数据类型转换有三种基本方法：

- 使用astype()函数进行强制类型转换，比如：dataset['col3'].astype('float')
- 自定义函数进行数据类型转换
- 使用Pandas提供的函数如to_numeric()、to_datetime()

当待转换列中含有不能转换的特殊值时(例如￥,ErrorValue,14n等)astype()函数将失效。
astype()函数有效的情形：

- 数据列中的每一个单位都能简单的解释为数字(2, 2.12等）
- 数据列中的每一个单位都是数值类型且向字符串object类型转换

Pandas的astype()函数和复杂的自定函数之间有一个中间段，那就是Pandas的一些辅助函数。这些辅助函数对于某些特定数据类型的转换非常有用(如to_numeric()、to_datetime())。

#### 数据操作

包括旋转、连接等。

In [43]:
# 返回列数：
print(dataset.shape[1])
# 返回行数：
print(dataset.shape[0])
# factorize(values[, sort, order, …])	Encode the object as an enumerated type or categorical variable.
labels, uniques = pd.factorize(['b', 'b', 'a', 'c', 'b'])
print(labels)
uniques

3
3
[0 0 1 2 0]


array(['b', 'a', 'c'], dtype=object)

对dataframe数据执行批量运算，使用pandas的apply函数。

In [44]:
import pandas as pd
import numpy as np
df = pd.DataFrame([[4, 9]] * 3, columns=['A', 'B'])
print(df)
df.apply(np.sqrt)

   A  B
0  4  9
1  4  9
2  4  9


,A,B
0,2.0,3.0
1,2.0,3.0
2,2.0,3.0


In [45]:
# 作用到每列
df.apply(np.sum, axis=0)

A    12
B    27
dtype: int64

In [46]:
# 作用到每行
df.apply(lambda x: [1, 2], axis=1)

0    [1, 2]
1    [1, 2]
2    [1, 2]
dtype: object